In [1]:
import xarray as xr
import numpy as np

# Load the 4D SINMOD dataset
ds = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc")

# Check the dataset structure
print(ds)

<xarray.Dataset> Size: 157GB
Dimensions:          (time: 365, dummySampleDim: 6, zc: 50, xc: 950, yc: 555)
Coordinates:
    time             (time, dummySampleDim) int32 9kB ...
  * xc               (xc) float32 4kB 800.0 1.6e+03 ... 7.592e+05 7.6e+05
  * yc               (yc) float32 2kB 800.0 1.6e+03 ... 4.432e+05 4.44e+05
  * zc               (zc) float32 200B 3.0 6.0 10.0 ... 2.5e+03 2.75e+03 3e+03
Dimensions without coordinates: dummySampleDim
Data variables:
    grid_mapping     int32 4B ...
    LayerDepths      (zc) float32 200B ...
    depth            (yc, xc) float32 2MB ...
    DXxDYy           (yc, xc) float32 2MB ...
    u_velocity       (time, zc, yc, xc) float32 38GB ...
    v_velocity       (time, zc, yc, xc) float32 38GB ...
    elevation        (time, yc, xc) float32 770MB ...
    temperature      (time, zc, yc, xc) float32 38GB ...
    salinity         (time, zc, yc, xc) float32 38GB ...
    ice_thickness    (time, yc, xc) float32 770MB ...
    ice_compactness  (time

In [2]:
ds['temperature'][0, -1, 0, 0].isnull()

<xarray.DataArray 'temperature' ()> Size: 1B
array(True)
Coordinates:
    xc       float32 4B 800.0
    yc       float32 4B 800.0
    zc       float32 4B 3e+03

In [5]:
# Example variable to process (e.g., temperature)
data_var = "temperature"  # Replace with your variable name

# First stage: Create a valid mask for land/ocean based on the first time step
masked_data = ds[data_var][0]  # Use only the first time step
valid_mask = ~masked_data.isnull()  # Land points will be False

# Second stage: Mask ocean layers based on unmasked values
# Mask the 4D data using the valid ocean mask
ocean_data = ds[data_var].where(valid_mask)

# For each grid cell, find the index of the bottom-most layer (first dimension)
bottom_layer_idx = (~ocean_data.isnull()).argmax(dim="zc")


# Extract the data for the bottom layer across all time steps
#bottom_layer_data = ds[data_var].isel(layer=bottom_layer_idx)

# # Average the extracted bottom layer data over time
# time_avg_bottom_layer = bottom_layer_data.mean(dim="time")

# # Save the result to a new NetCDF file (optional)
# time_avg_bottom_layer.to_netcdf("time_avg_bottom_layer.nc")

# # Inspect the final result
# print(time_avg_bottom_layer)

: 

In [ ]:
lowest_layer_idx